In [2]:
from datetime import datetime, timedelta
import requests
import zipfile
import os
import pandas as pd
import glob
import time
t1 = time.time()
# from google.cloud import bigquery
# client = bigquery.Client()

# Create a UDF for selecting the most recent available link for CMS data publication

In [3]:
# The follwoing function accept the CMS PAC Archive apge and find the link to the most recent published line for NH, HHA, Hospice. ITF & LTACH!
def get_most_recent_url(base_url, pac):
    # Get the current year and month
    current_date = datetime.now().date()# datetime(2023, 1 , 1).date()
    current_year = current_date.strftime('%Y')
    current_month = current_date.strftime('%m')


    # Calculate the previous month and year - it is applicable for the january. If cms has not uploaded new dataset, we need to check the December of prior year.
    previous_month_date = current_date - timedelta(days=30)
    previous_month = previous_month_date.strftime("%m")
    previous_month_years = [f'{previous_month}_{current_year}']

    # Create a list of month-year combinations
    month_years = [f'{current_month}_{current_year}']

    # Check if current month is January and adjust the month-year combinations accordingly
    if current_month == '01':
        prev_year = str(int(current_year) - 1)
        month_years.append(f'{prev_year}_12')
    else:
        month_years.append(f'{str(int(current_month)-1).zfill(2)}_{current_year}')

    # Iterate through the month-year combinations to select the url for the most recent published data:
    for month_year in month_years:
        # Create the URL
        url = base_url + f'{current_year}/{pac}_{month_year}.zip'

        # Check if the URL exists
        response = requests.head(url)
        if response.status_code == 200:
            # print(f"URL found: {url}")
            break
    else:
        url = base_url + f'{current_year}/{pac}_{previous_month_years}.zip'   # In January we may need to check this part!!!!
    return url



## SNF -  CMS Data

### Find the most recent URL for SNF data from cms datasets
https://data.cms.gov/provider-data/archived-data/nursing-homes


In [4]:
# Introducinthe base url for NH datasets
NH_base_url = 'https://data.cms.gov/provider-data/sites/default/files/archive/Nursing%20homes%20including%20rehab%20services/'
pac = 'nursing_homes_including_rehab_services'
# Use the UDF to generate the most recent URL
NH_url = get_most_recent_url(NH_base_url, pac)
print(f"final URL found: {NH_url}")


final URL found: https://data.cms.gov/provider-data/sites/default/files/archive/Nursing%20homes%20including%20rehab%20services/2023/nursing_homes_including_rehab_services_06_2023.zip


### Unzip the folder and prepare files for processing

In [5]:
%%time

# Download the file
response = requests.get(NH_url)
# Save the file locally
with open(r"NH_file.zip", "wb") as file:
    file.write(response.content)

# Unzip the file
with zipfile.ZipFile("NH_file.zip", "r") as zip_ref:
    zip_ref.extractall("NH_unzipped_folder")

CPU times: total: 1.42 s
Wall time: 10.4 s


### Process the NH_ProviderInfo

In [6]:
# To check if a path exist
# os.path.exists('file.zip')

In [7]:
# Create a dictionary to rename columns to be match with GCP tables
new_column_names_NH_ProviderInfo = {
"Federal Provider Number":"Federal_Provider_Number",
"Provider Name":"Provider_Name",
"Provider Address":"Provider_Address",
"Provider City":"Provider_City",
"Provider State":"Provider_State",
"Provider Zip Code":"Provider_Zip_Code",
"Provider Phone Number":"Provider_Phone_Number",
"Provider SSA County Code":"Provider_SSA_County_Code",
"Provider County Name":"Provider_County_Name",
"Ownership Type":"Ownership_Type",
"Number of Certified Beds":"Number_of_Certified_Beds",
"Average Number of Residents per Day":"Average_Number_of_Residents_per_Day",
"Average Number of Residents per Day Footnote":"Average_Number_of_Residents_per_Day_Footnote",
"Provider Type":"Provider_Type",
"Provider Resides in Hospital":"Provider_Resides_in_Hospital",
"Legal Business Name":"Legal_Business_Name",
"Date First Approved to Provide Medicare and Medicaid Services":"Date_First_Approved_to_Provide_Medicare_and_Medicaid_Services",
"Continuing Care Retirement Community":"Continuing_Care_Retirement_Community",
"Special Focus Status":"Special_Focus_Status",
"Abuse Icon":"Abuse_Icon",
"Most Recent Health Inspection More Than 2 Years Ago":"Most_Recent_Health_Inspection_More_Than_2_Years_Ago",
"Provider Changed Ownership in Last 12 Months":"Provider_Changed_Ownership_in_Last_12_Months",
"With a Resident and Family Council":"With_a_Resident_and_Family_Council",
"Automatic Sprinkler Systems in All Required Areas":"Automatic_Sprinkler_Systems_in_All_Required_Areas",
"Overall Rating":"Overall_Rating",
"Overall Rating Footnote":"Overall_Rating_Footnote",
"Health Inspection Rating":"Health_Inspection_Rating",
"Health Inspection Rating Footnote":"Health_Inspection_Rating_Footnote",
"QM Rating":"QM_Rating",
"QM Rating Footnote":"QM_Rating_Footnote",
"Long-Stay QM Rating":"Long_Stay_QM_Rating",
"Long-Stay QM Rating Footnote":"Long_Stay_QM_Rating_Footnote",
"Short-Stay QM Rating":"Short_Stay_QM_Rating",
"Short-Stay QM Rating Footnote":"Short_Stay_QM_Rating_Footnote",
"Staffing Rating":"Staffing_Rating",
"Staffing Rating Footnote":"Staffing_Rating_Footnote",
"Reported Staffing Footnote":"Reported_Staffing_Footnote",
"Physical Therapist Staffing Footnote":"Physical_Therapist_Staffing_Footnote",
"Reported Nurse Aide Staffing Hours per Resident per Day":"Reported_Nurse_Aide_Staffing_Hours_per_Resident_per_Day",
"Reported LPN Staffing Hours per Resident per Day":"Reported_LPN_Staffing_Hours_per_Resident_per_Day",
"Reported RN Staffing Hours per Resident per Day":"Reported_RN_Staffing_Hours_per_Resident_per_Day",
"Reported Licensed Staffing Hours per Resident per Day":"Reported_Licensed_Staffing_Hours_per_Resident_per_Day",
"Reported Total Nurse Staffing Hours per Resident per Day":"Reported_Total_Nurse_Staffing_Hours_per_Resident_per_Day",
"Total number of nurse staff hours per resident per day on the weekend":"Total_number_of_nurse_staff_hours_per_resident_per_day_on_the_weekend",
"Registered Nurse hours per resident per day on the weekend":"Registered_Nurse_hours_per_resident_per_day_on_the_weekend",
"Reported Physical Therapist Staffing Hours per Resident Per Day":"Reported_Physical_Therapist_Staffing_Hours_per_Resident_Per_Day",
"Total nursing staff turnover":"Total_nursing_staff_turnover",
"Total nursing staff turnover footnote":"Total_nursing_staff_turnover_footnote",
"Registered Nurse turnover":"Registered_Nurse_turnover",
"Registered Nurse turnover footnote":"Registered_Nurse_turnover_footnote",
"Number of administrators who have left the nursing home":"Number_of_administrators_who_have_left_the_nursing_home",
"Administrator turnover footnote":"Administrator_turnover_footnote",
"Case-Mix Nurse Aide Staffing Hours per Resident per Day":"Case_Mix_Nurse_Aide_Staffing_Hours_per_Resident_per_Day",
"Case-Mix LPN Staffing Hours per Resident per Day":"Case_Mix_LPN_Staffing_Hours_per_Resident_per_Day",
"Case-Mix RN Staffing Hours per Resident per Day":"Case_Mix_RN_Staffing_Hours_per_Resident_per_Day",
"Case-Mix Total Nurse Staffing Hours per Resident per Day":"Case_Mix_Total_Nurse_Staffing_Hours_per_Resident_per_Day",
"Adjusted Nurse Aide Staffing Hours per Resident per Day":"Adjusted_Nurse_Aide_Staffing_Hours_per_Resident_per_Day",
"Adjusted LPN Staffing Hours per Resident per Day":"Adjusted_LPN_Staffing_Hours_per_Resident_per_Day",
"Adjusted RN Staffing Hours per Resident per Day":"Adjusted_RN_Staffing_Hours_per_Resident_per_Day",
"Adjusted Total Nurse Staffing Hours per Resident per Day":"Adjusted_Total_Nurse_Staffing_Hours_per_Resident_per_Day",
"Adjusted Weekend Total Nurse Staffing Hours per Resident per Day":"Adjusted_Weekend_Total_Nurse_Staffing_Hours_per_Resident_per_Day",
"Rating Cycle 1 Standard Survey Health Date":"Rating_Cycle_1_Standard_Survey_Health_Date",
"Rating Cycle 1 Total Number of Health Deficiencies":"Rating_Cycle_1_Total_Number_of_Health_Deficiencies",
"Rating Cycle 1 Number of Standard Health Deficiencies":"Rating_Cycle_1_Number_of_Standard_Health_Deficiencies",
"Rating Cycle 1 Number of Complaint Health Deficiencies":"Rating_Cycle_1_Number_of_Complaint_Health_Deficiencies",
"Rating Cycle 1 Health Deficiency Score":"Rating_Cycle_1_Health_Deficiency_Score",
"Rating Cycle 1 Number of Health Revisits":"Rating_Cycle_1_Number_of_Health_Revisits",
"Rating Cycle 1 Health Revisit Score":"Rating_Cycle_1_Health_Revisit_Score",
"Rating Cycle 1 Total Health Score":"Rating_Cycle_1_Total_Health_Score",
"Rating Cycle 2 Standard Health Survey Date":"Rating_Cycle_2_Standard_Health_Survey_Date",
"Rating Cycle 2 Total Number of Health Deficiencies":"Rating_Cycle_2_Total_Number_of_Health_Deficiencies",
"Rating Cycle 2 Number of Standard Health Deficiencies":"Rating_Cycle_2_Number_of_Standard_Health_Deficiencies",
"Rating Cycle 2 Number of Complaint Health Deficiencies":"Rating_Cycle_2_Number_of_Complaint_Health_Deficiencies",
"Rating Cycle 2 Health Deficiency Score":"Rating_Cycle_2_Health_Deficiency_Score",
"Rating Cycle 2 Number of Health Revisits":"Rating_Cycle_2_Number_of_Health_Revisits",
"Rating Cycle 2 Health Revisit Score":"Rating_Cycle_2_Health_Revisit_Score",
"Rating Cycle 2 Total Health Score":"Rating_Cycle_2_Total_Health_Score",
"Rating Cycle 3 Standard Health Survey Date":"Rating_Cycle_3_Standard_Health_Survey_Date",
"Rating Cycle 3 Total Number of Health Deficiencies":"Rating_Cycle_3_Total_Number_of_Health_Deficiencies",
"Rating Cycle 3 Number of Standard Health Deficiencies":"Rating_Cycle_3_Number_of_Standard_Health_Deficiencies",
"Rating Cycle 3 Number of Complaint Health Deficiencies":"Rating_Cycle_3_Number_of_Complaint_Health_Deficiencies",
"Rating Cycle 3 Health Deficiency Score":"Rating_Cycle_3_Health_Deficiency_Score",
"Rating Cycle 3 Number of Health Revisits":"Rating_Cycle_3_Number_of_Health_Revisits",
"Rating Cycle 3 Health Revisit Score":"Rating_Cycle_3_Health_Revisit_Score",
"Rating Cycle 3 Total Health Score":"Rating_Cycle_3_Total_Health_Score",
"Total Weighted Health Survey Score":"Total_Weighted_Health_Survey_Score",
"Number of Facility Reported Incidents":"Number_of_Facility_Reported_Incidents",
"Number of Substantiated Complaints":"Number_of_Substantiated_Complaints",
"Number of Citations from Infection Control Inspections":"Number_of_Citations_from_Infection_Control_Inspections",
"Number of Fines":"Number_of_Fines",
"Total Amount of Fines in Dollars":"Total_Amount_of_Fines_in_Dollars",
"Number of Payment Denials":"Number_of_Payment_Denials",
"Total Number of Penalties":"Total_Number_of_Penalties",
"Location":"Location",
"Processing Date":"Processing_Date"
}


# Read the file: Since the Month_year of in the file name will be changed in each month, this code helps to have a wildcard system to read the NH_ProviderInfo regardless of the monthYear.
wildcard_pattern = r'NH_unzipped_folder/NH_ProviderInfo_*.csv'

file_path = glob.glob(wildcard_pattern)[0]
NH_ProviderInfo = pd.read_csv(file_path, index_col=None, dtype=str)

# Rename columns
NH_ProviderInfo = NH_ProviderInfo.rename(columns=new_column_names_NH_ProviderInfo)

# add the date for the load Load_timestamp, named Last_update
NH_ProviderInfo["Last_update"] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))

#Test the data
NH_ProviderInfo.head(2)

,Federal_Provider_Number,Provider_Name,Provider_Address,Provider_City,Provider_State,Provider_Zip_Code,Provider_Phone_Number,Provider_SSA_County_Code,Provider_County_Name,Ownership_Type,...,Number_of_Facility_Reported_Incidents,Number_of_Substantiated_Complaints,Number_of_Citations_from_Infection_Control_Inspections,Number_of_Fines,Total_Amount_of_Fines_in_Dollars,Number_of_Payment_Denials,Total_Number_of_Penalties,Location,Processing_Date,Last_update
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,290,Franklin,For profit - Corporation,...,2,0,0,1,655.14,0,1,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",2023-05-01,2023-06-08
1,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,For profit - Corporation,...,0,0,0,1,650.00,0,1,"260 WEST WALNUT STREET,SYLACAUGA,AL,35150",2023-05-01,2023-06-08


In [7]:
%%time

# save the processed data ingcp table
# client = bigquery.Client(location="US", project="asc-ahnat-adsi-dev")
# dataset_id = 'post_acute_conform'
client = bigquery.Client(location="US", project="xxx-sandbox")    #Should remove in final version
dataset_id = 'shahabi_davood' 
dataset = client.dataset(dataset_id)
table_ref = dataset.table('cms_nh_providerinfo_land')
try:
    job = client.delete_table('{}.{}'.format(dataset_id, table_ref))
except:
    print("Table could not be deleted. Probably because it doesn't exist")
                              
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
job = client.load_table_from_dataframe(NH_ProviderInfo, table_ref, location="US",job_config=job_config)
job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))
print(pd.Timestamp.now())

Table could not be deleted. Probably because it doesn't exist
Loaded dataframe to /projects/asc-ahnat-adsi-sandbox/datasets/shahabi_davood/tables/cms_nh_providerinfo_land
2023-06-08 10:25:31.411580
CPU times: user 157 ms, sys: 20.6 ms, total: 178 ms
Wall time: 6.29 s


### Process the NH_QualityMsr_Claims

In [8]:
# Rename columns
new_column_names_NH_QualityMsr_Claims = {
"Federal Provider Number":"Federal_Provider_Number",
"Provider Name":"Provider_Name",
"Provider Address":"Provider_Address",
"Provider City":"Provider_City",
"Provider State":"Provider_State",
"Provider Zip Code":"Provider_Zip_Code",
"Measure Code":"Measure_Code",
"Measure Description":"Measure_Description",
"Resident type":"Resident_type",
"Adjusted Score":"Adjusted_Score",
"Observed Score":"Observed_Score",
"Expected Score":"Expected_Score",
"Footnote for Score":"Footnote_for_Score",
"Used in Quality Measure Five Star Rating":"Used_in_Quality_Measure_Five_Star_Rating",
"Measure Period":"Measure_Period",
"Location":"Location",
"Processing Date":"Processing_Date"
}

# Read the file: Since the Month_year of in the file name will be changed in each month, this code helps to have a wildcard system to read the NH_QualityMsr_Claims regardless of the monthYear.
wildcard_pattern = r'NH_unzipped_folder/NH_QualityMsr_Claims_*.csv'
file_path = glob.glob(wildcard_pattern)[0]
NH_QualityMsr_Claims = pd.read_csv(file_path, index_col=None, dtype=str)

# Rename columns
NH_QualityMsr_Claims = NH_QualityMsr_Claims.rename(columns=new_column_names_NH_ProviderInfo)
NH_QualityMsr_Claims["Last_update"] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))

#Test the data
NH_QualityMsr_Claims.head(2)

,Federal_Provider_Number,Provider_Name,Provider_Address,Provider_City,Provider_State,Provider_Zip_Code,Measure Code,Measure Description,Resident type,Adjusted Score,Observed Score,Expected Score,Footnote for Score,Used in Quality Measure Five Star Rating,Measure Period,Location,Processing_Date,Last_update
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,521,Percentage of short-stay residents who were re...,Short Stay,13.789776,14.285714,23.428980,NaN,Y,20211001-20220930,"701 MONROE STREET NW, RUSSELLVILLE, AL, 35653",2023-05-01,2023-06-08
1,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,522,Percentage of short-stay residents who had an ...,Short Stay,4.693847,4.761905,10.975152,NaN,Y,20211001-20220930,"701 MONROE STREET NW, RUSSELLVILLE, AL, 35653",2023-05-01,2023-06-08


In [9]:
# client = bigquery.Client(location="US", project="asc-ahnat-adsi-dev")
# dataset_id = 'post_acute_conform'
client = bigquery.Client(location="US", project="xxx-sandbox")  #Should remove in final version
dataset_id = 'shahabi_davood'      #Should remove in final version
dataset = client.dataset(dataset_id)
table_ref = dataset.table('cms_nh_qualitymsr_claims_land')
try:
    job = client.delete_table('{}.{}'.format(dataset_id, table_ref))
except:
    print("Table could not be deleted. Probably because it doesn't exist")
                              
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
job = client.load_table_from_dataframe(NH_QualityMsr_Claims, table_ref, location="US",job_config=job_config)
job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))
print(pd.Timestamp.now())

Table could not be deleted. Probably because it doesn't exist
Loaded dataframe to /projects/asc-ahnat-adsi-sandbox/datasets/shahabi_davood/tables/cms_nh_qualitymsr_claims_land
2023-06-08 10:25:34.452085


# HHA -  CMS Data
### Find the most recent URL for HHA data from cms datasets
https://data.cms.gov/provider-data/archived-data/home-health-services

In [10]:
# Introducinthe base url for NH datasets
HH_base_url = 'https://data.cms.gov/provider-data/sites/default/files/archive/Home%20health%20services/'

pac = 'home_health_services'
# Use the UDF to generate the most recent URL
HH_url = get_most_recent_url(HH_base_url,pac)
print(f"final URL found: {HH_url}")


final URL found: https://data.cms.gov/provider-data/sites/default/files/archive/Home%20health%20services/2023/home_health_services_05_2023.zip


### Unzip the folder and prepare files for processing

In [11]:
%%time

# Download the file
response = requests.get(HH_url)
# Save the file locally
with open(r"HH_file.zip", "wb") as file:
    file.write(response.content)

# Unzip the file
with zipfile.ZipFile("HH_file.zip", "r") as zip_ref:
    zip_ref.extractall("HH_unzipped_folder")

CPU times: user 64.1 ms, sys: 24.2 ms, total: 88.3 ms
Wall time: 395 ms


### Process the HH_ProviderInfo

In [12]:
# Create a dictionary to rename columns to be match with GCP tables
new_column_names_HH_ProviderInfo = {
"State":"State",
"CMS Certification Number (CCN)":"CMS_Certification_Number_CCN",
"Provider Name":"Provider_Name",
"Address":"Address",
"City":"City",
"ZIP":"ZIP",
"Phone":"Phone",
"Type of Ownership":"Type_of_Ownership",
"Offers Nursing Care Services":"Offers_Nursing_Care_Services",
"Offers Physical Therapy Services":"Offers_Physical_Therapy_Services",
"Offers Occupational Therapy Services":"Offers_Occupational_Therapy_Services",
"Offers Speech Pathology Services":"Offers_Speech_Pathology_Services",
"Offers Medical Social Services":"Offers_Medical_Social_Services",
"Offers Home Health Aide Services":"Offers_Home_Health_Aide_Services",
"Date Certified":"Date_Certified",
"Quality of patient care star rating":"Quality_of_patient_care_star_rating",
"Footnote for quality of patient care star rating":"Footnote_for_quality_of_patient_care_star_rating",
"How often the home health team began their patients' care in a timely manner":"How_often_the_home_health_team_began_their_patients_care_in_a_timely_manner",
"Footnote for how often the home health team began their patients' care in a timely manner":"Footnote_for_how_often_the_home_health_team_began_their_patients_care_in_a_timely_manner",
"How often the home health team taught patients (or their family caregivers) about their drugs":"How_often_the_home_health_team_taught_patients_or_their_family_caregivers_about_their_drugs",
"Footnote for how often the home health team taught patients (or their family caregivers) about their drugs":"Footnote_for_how_often_the_home_health_team_taught_patients_or_their_family_caregivers_about_their_drugs",
"How often the home health team determined whether patients received a flu shot for the current flu season":"How_often_the_home_health_team_determined_whether_patients_received_a_flu_shot_for_the_current_flu_season",
"Footnote for how often the home health team determined whether patients received a flu shot for the current flu season":"Footnote_for_how_often_the_home_health_team_determined_whether_patients_received_a_flu_shot_for_the_current_flu_season",
"How often patients got better at walking or moving around":"How_often_patients_got_better_at_walking_or_moving_around",
"Footnote for how often patients got better at walking or moving around":"Footnote_for_how_often_patients_got_better_at_walking_or_moving_around",
"How often patients got better at getting in and out of bed":"How_often_patients_got_better_at_getting_in_and_out_of_bed",
"Footnote for how often patients got better at getting in and out of bed":"Footnote_for_how_often_patients_got_better_at_getting_in_and_out_of_bed",
"How often patients got better at bathing":"How_often_patients_got_better_at_bathing",
"Footnote for how often patients got better at bathing":"Footnote_for_how_often_patients_got_better_at_bathing",
"How often patients' breathing improved":"How_often_patients_breathing_improved",
"Footnote for how often patients' breathing improved":"Footnote_for_how_often_patients_breathing_improved",
"How often patients got better at taking their drugs correctly by mouth":"How_often_patients_got_better_at_taking_their_drugs_correctly_by_mouth",
"Footnote for how often patients got better at taking their drugs correctly by mouth":"Footnote_for_how_often_patients_got_better_at_taking_their_drugs_correctly_by_mouth",
"How often home health patients had to be admitted to the hospital":"How_often_home_health_patients_had_to_be_admitted_to_the_hospital",
"Footnote for how often home health patients had to be admitted to the hospital":"Footnote_for_how_often_home_health_patients_had_to_be_admitted_to_the_hospital",
"How often patients receiving home health care needed urgent, unplanned care in the ER without being admitted":"How_often_patients_receiving_home_health_care_needed_urgent_unplanned_care_in_the_ER_without_being_admitted",
"Footnote for how often patients receiving home health care needed urgent, unplanned care in the ER without being admitted":"Footnote_for_how_often_patients_receiving_home_health_care_needed_urgent_unplanned_care_in_the_ER_without_being_admitted",
"Changes in skin integrity post-acute care: pressure ulcer/injury":"Changes_in_skin_integrity_post_acute_care_pressure_ulcer_injury",
"Footnote Changes in skin integrity post-acute care: pressure ulcer/injury":"Footnote_Changes_in_skin_integrity_post_acute_care_pressure_ulcer_injury",
"How often physician-recommended actions to address medication issues were completely timely":"How_often_physician_recommended_actions_to_address_medication_issues_were_completely_timely",
"Footnote for how often physician-recommended actions to address medication issues were completely timely":"Footnote_for_how_often_physician_recommended_actions_to_address_medication_issues_were_completely_timely",
"Percent of Residents Experiencing One or More Falls with Major Injury":"Percent_of_Residents_Experiencing_One_or_More_Falls_with_Major_Injury",
"Footnote for Percent of Residents Experiencing One or More Falls with Major Injury":"Footnote_for_Percent_of_Residents_Experiencing_One_or_More_Falls_with_Major_Injury",
"Application of Percent of Long Term Care Hospital Patients with an Admission and Discharge Functional Assessment and a Care Plan that Addresses Function":"Application_of_Percent_of_Long_Term_Care_Hospital_Patients_with_an_Admission_and_Discharge_Functional_Assessment_and_a_Care_Plan_that_Addresses_Function",
"Footnote for Application of Percent of Long Term Care Hospital Patients with an Admission and Discharge Functional Assessment and a Care Plan that Addresses Function":"Footnote_for_Application_of_Percent_of_Long_Term_Care_Hospital_Patients_with_an_Admission_and_Discharge_Functional_Assessment_and_a_Care_Plan_that_Addresses_Function",
"DTC Numerator":"DTC_Numerator",
"DTC Denominator":"DTC_Denominator",
"DTC Observed Rate":"DTC_Observed_Rate",
"DTC Risk-Standardized Rate":"DTC_Risk_Standardized_Rate",
"DTC Risk-Standardized Rate (Lower Limit)":"DTC_Risk_Standardized_Rate_Lower_Limit",
"DTC Risk-Standardized Rate (Upper Limit)":"DTC_Risk_Standardized_Rate_Upper_Limit",
"DTC Performance Categorization":"DTC_Performance_Categorization",
"Footnote for DTC Risk-Standardized Rate":"Footnote_for_DTC_Risk_Standardized_Rate",
"PPR Numerator":"PPR_Numerator",
"PPR Denominator":"PPR_Denominator",
"PPR Observed Rate":"PPR_Observed_Rate",
"PPR Risk-Standardized Rate":"PPR_Risk_Standardized_Rate",
"PPR Risk-Standardized Rate (Lower Limit)":"PPR_Risk_Standardized_Rate_Lower_Limit",
"PPR Risk-Standardized Rate (Upper Limit)":"PPR_Risk_Standardized_Rate_Upper_Limit",
"PPR Performance Categorization":"PPR_Performance_Categorization",
"Footnote for PPR Risk-Standardized Rate":"Footnote_for_PPR_Risk_Standardized_Rate",
"How much Medicare spends on an episode of care at this agency, compared to Medicare spending across all agencies nationally":"How_much_Medicare_spends_on_an_episode_of_care_at_this_agency_compared_to_Medicare_spending_across_all_agencies_nationally",
"Footnote for how much Medicare spends on an episode of care at this agency, compared to Medicare spending across all agencies nationally":"Footnote_for_how_much_Medicare_spends_on_an_episode_of_care_at_this_agency_compared_to_Medicare_spending_across_all_agencies_nationally",
"No. of episodes to calc how much Medicare spends per episode of care at agency, compared to spending at all agencies (national)":"No_of_episodes_to_calc_how_much_Medicare_spends_per_episode_of_care_at_agency_compared_to_spending_at_all_agencies_national"
}


# Read the file: Since the Month_year of in the file name will be changed in each month, this code helps to have a wildcard system to read the NH_ProviderInfo regardless of the monthYear.
wildcard_pattern = r'HH_unzipped_folder/HH_Provider_*.csv'

file_path = glob.glob(wildcard_pattern)[0]
HH_ProviderInfo = pd.read_csv(file_path, index_col=None, dtype=str)

# Rename columns
HH_ProviderInfo = NH_ProviderInfo.rename(columns=new_column_names_HH_ProviderInfo)

# add the date for the load Load_timestamp, named Last_update
HH_ProviderInfo["Last_update"] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))

#Test the data
HH_ProviderInfo.head(2)

,Federal_Provider_Number,Provider_Name,Provider_Address,Provider_City,Provider_State,Provider_Zip_Code,Provider_Phone_Number,Provider_SSA_County_Code,Provider_County_Name,Ownership_Type,...,Number_of_Facility_Reported_Incidents,Number_of_Substantiated_Complaints,Number_of_Citations_from_Infection_Control_Inspections,Number_of_Fines,Total_Amount_of_Fines_in_Dollars,Number_of_Payment_Denials,Total_Number_of_Penalties,Location,Processing_Date,Last_update
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,290,Franklin,For profit - Corporation,...,2,0,0,1,655.14,0,1,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",2023-05-01,2023-06-08
1,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,For profit - Corporation,...,0,0,0,1,650.00,0,1,"260 WEST WALNUT STREET,SYLACAUGA,AL,35150",2023-05-01,2023-06-08


In [13]:
%%time

# save the processed data ingcp table
# client = bigquery.Client(location="US", project="asc-ahnat-adsi-dev")
# dataset_id = 'post_acute_conform'
client = bigquery.Client(location="US", project="xxx-sandbox")    #Should remove in final version
dataset_id = 'shahabi_davood' 
dataset = client.dataset(dataset_id)
table_ref = dataset.table('cms_hh_providerinfo_land')
try:
    job = client.delete_table('{}.{}'.format(dataset_id, table_ref))
except:
    print("Table could not be deleted. Probably because it doesn't exist")
                              
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
job = client.load_table_from_dataframe(HH_ProviderInfo, table_ref, location="US",job_config=job_config)
job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))
print(pd.Timestamp.now())

Table could not be deleted. Probably because it doesn't exist
Loaded dataframe to /projects/asc-ahnat-adsi-sandbox/datasets/shahabi_davood/tables/cms_hh_providerinfo_land
2023-06-08 10:25:47.435388
CPU times: user 157 ms, sys: 15.6 ms, total: 173 ms
Wall time: 9.47 s


### Process the HHCAHPS_Provider

In [14]:
# Create a dictionary to rename columns to be match with GCP tables
new_column_names_HHCAHPS_Provider = {
"CMS Certification Number (CCN)":"CMS_Certification_Number_CCN",
"HHCAHPS Survey Summary Star Rating":"HHCAHPS_Survey_Summary_Star_Rating",
"HHCAHPS Survey Summary Star Rating Footnote":"HHCAHPS_Survey_Summary_Star_Rating_Footnote",
"Star Rating for health team gave care in a professional way":"Star_Rating_for_health_team_gave_care_in_a_professional_way",
"Footnote for Star Rating for gave care in a professional way":"Footnote_for_Star_Rating_for_gave_care_in_a_professional_way",
"Percent of patients who reported that their home health team gave care in a professional way":"Percent_of_patients_who_reported_that_their_home_health_team_gave_care_in_a_professional_way",
"Footnote for Percent of patients who reported that their home health team gave care in a professional way":"Footnote_for_Percent_of_patients_who_reported_that_their_home_health_team_gave_care_in_a_professional_way",
"Star Rating for health team communicated well with them":"Star_Rating_for_health_team_communicated_well_with_them",
"Footnote for Star Rating for communicated well with them":"Footnote_for_Star_Rating_for_communicated_well_with_them",
"Percent of patients who reported that their home health team communicated well with them":"Percent_of_patients_who_reported_that_their_home_health_team_communicated_well_with_them",
"Footnote for Percent of patients who reported that their home health team communicated well with them":"Footnote_for_Percent_of_patients_who_reported_that_their_home_health_team_communicated_well_with_them",
"Star Rating team discussed medicines, pain, and home safety":"Star_Rating_team_discussed_medicines_pain_and_home_safety",
"Footnote Star Rating discussed medicines, pain, home safety":"Footnote_Star_Rating_discussed_medicines_pain_home_safety",
"Percent of patients who reported that their home health team discussed medicines, pain, and home safety with them":"Percent_of_patients_who_reported_that_their_home_health_team_discussed_medicines_pain_and_home_safety_with_them",
"Footnote for Percent of patients who reported that their home health team discussed medicines, pain, and home safety with them":"Footnote_for_Percent_of_patients_who_reported_that_their_home_health_team_discussed_medicines_pain_and_home_safety_with_them",
"Star Rating for how patients rated overall care from agency":"Star_Rating_for_how_patients_rated_overall_care_from_agency",
"Footnote for Star Rating for overall care from agency":"Footnote_for_Star_Rating_for_overall_care_from_agency",
"Percent of patients who gave their home health agency a rating of 9 or 10 on a scale from 0 (lowest) to 10 (highest)":"Percent_of_patients_who_gave_their_home_health_agency_a_rating_of_9_or_10_on_a_scale_from_0_lowest_to_10_highest",
"Footnote for Percent of patients who gave their home health agency a rating of 9 or 10 on a scale from 0(lowest) to 10(highest)":"Footnote_for_Percent_of_patients_who_gave_their_home_health_agency_a_rating_of_9_or_10_on_a_scale_from_0lowest_to_10highest",
"Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family":"Percent_of_patients_who_reported_YES_they_would_definitely_recommend_the_home_health_agency_to_friends_and_family",
"Footnote for Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family":"Footnote_for_Percent_of_patients_who_reported_YES_they_would_definitely_recommend_the_home_health_agency_to_friends_and_family",
"Number of completed Surveys":"Number_of_completed_Surveys",
"Footnote for number of completed surveys":"Footnote_for_number_of_completed_surveys",
"Response rate":"Response_rate",
"Footnote for response rate":"Footnote_for_response_rate",
"Footnote Number":"Footnote_Number"}


# Read the file: Since the Month_year of in the file name will be changed in each month, this code helps to have a wildcard system to read the NH_ProviderInfo regardless of the monthYear.
wildcard_pattern = r'HH_unzipped_folder/HHCAHPS_Provider_*.csv'

file_path = glob.glob(wildcard_pattern)[0]
HHCAHPS_Provider = pd.read_csv(file_path, index_col=None, dtype=str)

# Rename columns
HHCAHPS_Provider = HHCAHPS_Provider.rename(columns=new_column_names_HHCAHPS_Provider)

# add the date for the load Load_timestamp, named Last_update
HHCAHPS_Provider["Last_update"] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))

#Test the data
HHCAHPS_Provider.head(2)

,CMS_Certification_Number_CCN,HHCAHPS_Survey_Summary_Star_Rating,HHCAHPS_Survey_Summary_Star_Rating_Footnote,Star_Rating_for_health_team_gave_care_in_a_professional_way,Footnote_for_Star_Rating_for_gave_care_in_a_professional_way,Percent_of_patients_who_reported_that_their_home_health_team_gave_care_in_a_professional_way,Footnote_for_Percent_of_patients_who_reported_that_their_home_health_team_gave_care_in_a_professional_way,Star_Rating_for_health_team_communicated_well_with_them,Footnote_for_Star_Rating_for_communicated_well_with_them,Percent_of_patients_who_reported_that_their_home_health_team_communicated_well_with_them,...,Percent_of_patients_who_gave_their_home_health_agency_a_rating_of_9_or_10_on_a_scale_from_0_lowest_to_10_highest,Footnote_for_Percent_of_patients_who_gave_their_home_health_agency_a_rating_of_9_or_10_on_a_scale_from_0lowest_to_10highest,Percent_of_patients_who_reported_YES_they_would_definitely_recommend_the_home_health_agency_to_friends_and_family,Footnote_for_Percent_of_patients_who_reported_YES_they_would_definitely_recommend_the_home_health_agency_to_friends_and_family,Number_of_completed_Surveys,Footnote_for_number_of_completed_surveys,Response_rate,Footnote_for_response_rate,Footnote_Number,Last_update
0,017000,5,NaN,4,NaN,92,NaN,5,NaN,89,...,93,NaN,86,NaN,686,NaN,22,NaN,NaN,2023-06-08
1,017009,4,NaN,4,NaN,92,NaN,5,NaN,92,...,91,NaN,86,NaN,274,NaN,33,NaN,NaN,2023-06-08


In [15]:
%%time

# save the processed data ingcp table
# client = bigquery.Client(location="US", project="asc-ahnat-adsi-dev")
# dataset_id = 'post_acute_conform'
client = bigquery.Client(location="US", project="xxx-sandbox")    #Should remove in final version
dataset_id = 'shahabi_davood' 
dataset = client.dataset(dataset_id)
table_ref = dataset.table('cms_hhcahps_provider_land')
try:
    job = client.delete_table('{}.{}'.format(dataset_id, table_ref))
except:
    print("Table could not be deleted. Probably because it doesn't exist")
                              
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
job = client.load_table_from_dataframe(HHCAHPS_Provider, table_ref, location="US",job_config=job_config)
job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))
print(pd.Timestamp.now())

Table could not be deleted. Probably because it doesn't exist
Loaded dataframe to /projects/asc-ahnat-adsi-sandbox/datasets/shahabi_davood/tables/cms_hhcahps_provider_land
2023-06-08 10:25:52.432354
CPU times: user 54.2 ms, sys: 2.84 ms, total: 57.1 ms
Wall time: 4.95 s


# Hospice -  CMS Data
### Find the most recent URL for Hospice data from cms datasets
https://data.cms.gov/provider-data/archived-data/hospice-care

In [16]:
# Introducinthe base url for NH datasets
Hospice_base_url = 'https://data.cms.gov/provider-data/sites/default/files/archive/Hospice%20care/'
pac = 'hospice_care'
# Use the UDF to generate the most recent URL
Hospice_url = get_most_recent_url(Hospice_base_url, pac)
print(f"final URL found: {Hospice_url}")


final URL found: https://data.cms.gov/provider-data/sites/default/files/archive/Hospice%20care/2023/hospice_care_05_2023.zip


### Unzip the folder and prepare files for processing

In [17]:
%%time

# Download the file
response = requests.get(Hospice_url)
# Save the file locally
with open(r"Hospice_file.zip", "wb") as file:
    file.write(response.content)

# Unzip the file
with zipfile.ZipFile("Hospice_file.zip", "r") as zip_ref:
    zip_ref.extractall("Hospice_unzipped_folder")

CPU times: user 210 ms, sys: 88.6 ms, total: 298 ms
Wall time: 533 ms


### Process the Hospice_Provider

In [18]:
# Create a dictionary to rename columns to be match with GCP tables
new_column_names_Hospice_Provider = {
"CMS Certification Number (CCN)":"CMS_Certification_Number_CCN",
"Facility Name":"Facility_Name",
"Address Line 1":"Address_Line_1",
"Address Line 2":"Address_Line_2",
"City":"City",
"State":"State",
"Zip Code":"Zip_Code",
"County Name":"County_Name",
"Phone Number":"Phone_Number",
"CMS Region":"CMS_Region",
"Measure Code":"Measure_Code",
"Measure Name":"Measure_Name",
"Score":"Score",
"Footnote":"Footnote",
"Measure Date Range":"Measure_Date_Range"
}


# Read the file: Since the Month_year of in the file name will be changed in each month, this code helps to have a wildcard system to read the Hospice_Provider regardless of the monthYear.
wildcard_pattern = r'Hospice_unzipped_folder/Hospice_Provider_*.csv'

file_path = glob.glob(wildcard_pattern)[0]
Hospice_Provider = pd.read_csv(file_path, index_col=None, dtype=str,encoding='latin-1')

# Rename columns
Hospice_Provider = Hospice_Provider.rename(columns=new_column_names_Hospice_Provider)

# add the date for the load Load_timestamp, named Last_update
Hospice_Provider["Last_update"] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))

#Test the data
Hospice_Provider.head(2)

,CMS_Certification_Number_CCN,Facility_Name,Address_Line_1,Address_Line_2,City,State,Zip_Code,County_Name,Phone_Number,CMS_Region,Measure_Code,Measure_Name,Score,Footnote,Measure_Date_Range,Last_update
0,001500,AGAVE HOSPICE,3240 EAST UNION HILLS DRIVE SUITE 145 ROOM A,NaN,PHOENIX,AZ,85050,Maricopa,(602) 855-3500,9,H_001_01_DENOMINATOR,NaN,Not Available,"2,4",07/01/2021-06/30/2022,2023-06-08
1,001500,AGAVE HOSPICE,3240 EAST UNION HILLS DRIVE SUITE 145 ROOM A,NaN,PHOENIX,AZ,85050,Maricopa,(602) 855-3500,9,H_001_01_OBSERVED,Hospice and Palliative Care Treatment Preferences,Not Available,"2,4",07/01/2021-06/30/2022,2023-06-08


In [19]:
%%time

# save the processed data ingcp table
# client = bigquery.Client(location="US", project="asc-ahnat-adsi-dev")
# dataset_id = 'post_acute_conform'
client = bigquery.Client(location="US", project="xxx-sandbox")    #Should remove in final version
dataset_id = 'shahabi_davood' 
dataset = client.dataset(dataset_id)
table_ref = dataset.table('cms_hospice_providerinfo_land')
try:
    job = client.delete_table('{}.{}'.format(dataset_id, table_ref))
except:
    print("Table could not be deleted. Probably because it doesn't exist")
                              
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
job = client.load_table_from_dataframe(Hospice_Provider, table_ref, location="US",job_config=job_config)
job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))
print(pd.Timestamp.now())

Table could not be deleted. Probably because it doesn't exist


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/_pandas_helpers.py:472: UserWarning: Pyarrow could not determine the type of columns: Address_Line_2.
  ", ".join(field.name for field in unknown_type_fields)


Loaded dataframe to /projects/asc-ahnat-adsi-sandbox/datasets/shahabi_davood/tables/cms_hospice_providerinfo_land
2023-06-08 10:25:58.928035
CPU times: user 711 ms, sys: 40.8 ms, total: 751 ms
Wall time: 4.6 s
